# 🔍 Chapter 5 练习 3: 敏感性分析 - 因果结论的「压力测试」

## 无混淆假设能验证吗？

在因果推断中，有一个令人不安的事实：

> **无混淆假设 (Unconfoundedness) 无法从数据中验证**

我们永远不知道是否遗漏了某个重要的混淆因子！这就像你永远无法证明「房间里没有隐形的大象」。

那怎么办呢？**敏感性分析 (Sensitivity Analysis)** 来帮忙！

---

## 🎯 学习目标

1. 理解未观测混淆的影响
2. 实现 Rosenbaum 敏感性边界
3. **深入掌握 E-value 方法** ⭐
   - E-value 的概念和公式推导
   - 计算点估计和置信区间的 E-value
   - 解读和报告 E-value
   - 在实际研究中应用 E-value
4. 进行 Placebo 稳健性检验
5. 对比不同敏感性分析方法的优缺点

## 🌟 敏感性分析的直觉

### 类比：药物试验的「魔鬼代言人」

想象你是 FDA 的审查员，制药公司声称他们的新药有效：

- **他们说**: 「吃药后，患者恢复率高了 20%！」
- **你问**: 「会不会是因为选择吃药的人本来就更健康？」
- **他们说**: 「我们已经控制了所有观测到的因素！」
- **你问**: 「但如果有你们没测量的因素呢？比如患者的乐观程度？」

敏感性分析就是系统地问这个问题：

> **「需要多强的未观测混淆，才能推翻我们的结论？」**

### 两种方法

1. **Rosenbaum 敏感性边界**: 假设存在未观测混淆，看结论何时变得不显著
2. **E-value**: 计算能解释观测关联的最小混淆强度

## 📐 核心公式

### Rosenbaum 敏感性参数 Γ

对于两个协变量相同的个体 i 和 j：

$$\frac{1}{\Gamma} \leq \frac{P(T_i=1|X)}{P(T_j=1|X)} \leq \Gamma$$

- **Γ = 1**: 无未观测混淆（完美的随机化）
- **Γ = 2**: 倾向得分最多相差 2 倍
- **Γ = 3**: 倾向得分最多相差 3 倍

### E-value 公式 (VanderWeele & Ding, 2017)

**基本公式**:

$$E = RR + \sqrt{RR \times (RR - 1)}$$

其中 RR 是风险比 (Risk Ratio)。

**推导直觉**:
假设存在未观测混淆因子 U，它与处理 T 和结果 Y 的关联强度都是 RR_U。
E-value 就是使观测到的 RR 完全被 U 解释所需的最小 RR_U 值。

**重要性质**:
1. E-value ≥ 1 (最小值为 1，表示任何混淆都能推翻结论)
2. E-value 关于 RR 单调递增（效应越大，越稳健）
3. 当 RR = 1 (无效应)时，E-value = 1

**对于置信区间**:
- 点估计的 E-value: 基于 RR 的点估计
- CI 下界的 E-value: 基于置信区间下界（更保守的估计）

**实际解读**:
E-value 表示：**使观测关联完全被混淆解释所需的最小风险比**

例如，E-value = 3.0 意味着：
- 需要一个未观测因子 U
- U 使「接受处理的概率」提高 3 倍
- U 同时使「结果发生概率」提高 3 倍
- 才能完全解释掉观测到的效应

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict
from scipy import stats
from sklearn.linear_model import LinearRegression

# 设置随机种子
np.random.seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("环境配置完成! 🔍")

## 练习 3.1: 模拟未观测混淆

首先，让我们创建一个有**未观测混淆**的数据集，看看它如何影响因果估计！

### 场景

研究「参加培训」对「薪资」的影响：
- **X**: 观测到的能力指标
- **U**: 未观测的「动机」因素（我们假装看不到）
- **T**: 是否参加培训（受 X 和 U 影响）
- **Y**: 薪资（受 T、X、U 影响）

问题：高动机的人既更可能参加培训，也更可能有高薪资。

In [ ]:
def simulate_unobserved_confounding(
    n: int = 1000,
    confounder_strength: float = 0.5,
    seed: int = 42
) -> Tuple[pd.DataFrame, np.ndarray, Dict]:
    """
    模拟包含未观测混淆的数据
    
    数据生成过程:
    - X ~ N(0, 1)  # 观测协变量（能力）
    - U ~ N(0, 1)  # 未观测混淆因子（动机）
    - P(T=1|X,U) = sigmoid(0.5*X + strength*U)
    - Y = 10 + 2*T + 1.5*X + strength*2*U + noise
    
    真实 ATE = 2.0
    
    Returns:
        (df, U, params)
        df: 包含 X, T, Y 的 DataFrame (不包含 U!)
        U: 未观测混淆因子
        params: 真实参数
    """
    np.random.seed(seed)
    
    # TODO: 生成观测协变量 X (能力)
    X = None  # 你的代码: np.random.randn(n)
    
    # TODO: 生成未观测混淆因子 U (动机)
    U = None  # 你的代码: np.random.randn(n)
    
    # TODO: 生成处理 T (参加培训)
    # 高能力和高动机的人更可能参加培训
    # propensity_logit = 0.5*X + confounder_strength*U
    # propensity = sigmoid(propensity_logit) = 1 / (1 + exp(-logit))
    propensity_logit = None  # 你的代码
    propensity = None  # 你的代码
    T = None  # 你的代码: np.random.binomial(1, propensity)
    
    # TODO: 生成结果 Y (薪资)
    # Y = 10 + 2*T + 1.5*X + confounder_strength*2*U + noise
    # 解读: 培训真正增加薪资 2，能力影响 1.5，动机影响 confounder_strength*2
    noise = np.random.randn(n) * 0.5
    Y = None  # 你的代码
    
    # 创建 DataFrame (假装我们看不到 U!)
    df = pd.DataFrame({
        'X': X,
        'T': T,
        'Y': Y
    })
    
    params = {
        'true_ate': 2.0,
        'confounder_strength': confounder_strength
    }
    
    return df, U, params

In [ ]:
# 计算不同的 ATE 估计
def compute_naive_ate(df: pd.DataFrame) -> float:
    """
    朴素 ATE 估计: E[Y|T=1] - E[Y|T=0]
    (忽略所有混淆)
    """
    # TODO: 计算朴素估计
    # 你的代码
    pass


def compute_adjusted_ate(df: pd.DataFrame) -> float:
    """
    调整 X 后的 ATE 估计
    使用线性回归: Y ~ T + X
    """
    # TODO: 使用回归估计 ATE
    # 1. 构建特征矩阵 [[T1, X1], [T2, X2], ...]
    # 2. 拟合线性回归
    # 3. 返回 T 的系数
    
    # 你的代码
    pass

In [ ]:
# 测试不同混淆强度
print("=== 未观测混淆的影响 ===")
print(f"真实 ATE = 2.0\n")

strengths = [0.0, 0.3, 0.6, 1.0]
results = []

for strength in strengths:
    df, U, params = simulate_unobserved_confounding(
        n=2000,
        confounder_strength=strength
    )
    
    if df is not None and df['X'].iloc[0] is not None:
        naive_ate = compute_naive_ate(df)
        adjusted_ate = compute_adjusted_ate(df)
        
        if naive_ate is not None and adjusted_ate is not None:
            results.append({
                'Strength': strength,
                'Naive ATE': naive_ate,
                'Naive Bias': naive_ate - 2.0,
                'Adjusted ATE': adjusted_ate,
                'Adjusted Bias': adjusted_ate - 2.0
            })

if len(results) > 0:
    results_df = pd.DataFrame(results)
    display(results_df.round(4))
    
    print("\n解读:")
    print("- 混淆强度 = 0: 无混淆，两种估计都接近真实值")
    print("- 混淆强度增加: 朴素估计偏差增大（高估效应）")
    print("- 调整 X 后: 仍有偏差，因为 U 未被控制！")
else:
    print("[TODO] 请完成数据生成和估计函数")

In [ ]:
# 可视化混淆的影响
if len(results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 估计值 vs 混淆强度
    axes[0].plot(results_df['Strength'], results_df['Naive ATE'], 
                'bo-', label='朴素估计', linewidth=2, markersize=8)
    axes[0].plot(results_df['Strength'], results_df['Adjusted ATE'], 
                'gs-', label='调整 X 后', linewidth=2, markersize=8)
    axes[0].axhline(y=2.0, color='red', linestyle='--', label='真实 ATE')
    axes[0].set_xlabel('未观测混淆强度')
    axes[0].set_ylabel('ATE 估计')
    axes[0].set_title('估计值随混淆强度变化')
    axes[0].legend()
    
    # 偏差 vs 混淆强度
    axes[1].bar(np.arange(len(strengths)) - 0.2, results_df['Naive Bias'], 
               0.35, label='朴素估计偏差', color='#3498db')
    axes[1].bar(np.arange(len(strengths)) + 0.2, results_df['Adjusted Bias'], 
               0.35, label='调整后偏差', color='#2ecc71')
    axes[1].axhline(y=0, color='red', linestyle='--')
    axes[1].set_xticks(np.arange(len(strengths)))
    axes[1].set_xticklabels([f'{s:.1f}' for s in strengths])
    axes[1].set_xlabel('未观测混淆强度')
    axes[1].set_ylabel('偏差 (估计 - 真实)')
    axes[1].set_title('偏差随混淆强度变化')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 练习 3.2: Rosenbaum 敏感性边界

现在让我们实现 Rosenbaum 敏感性分析！

核心问题：**如果存在未观测混淆，我们的结论会有多脆弱？**

In [ ]:
def compute_rosenbaum_bounds(
    Y: np.ndarray,
    T: np.ndarray,
    gamma: float
) -> Tuple[float, float]:
    """
    计算 Rosenbaum 敏感性边界
    
    Γ 参数的含义:
    - Γ = 1: 假设无未观测混淆
    - Γ > 1: 允许倾向得分最多相差 Γ 倍
    
    Args:
        Y: 结果变量
        T: 处理变量
        gamma: 敏感性参数
    
    Returns:
        (lower_bound, upper_bound)
    """
    # 观测的 ATE
    ate_obs = Y[T == 1].mean() - Y[T == 0].mean()
    
    if gamma == 1.0:
        # 无混淆假设下，边界就是点估计
        return ate_obs, ate_obs
    
    # TODO: 计算敏感性边界
    # 简化版本: 边界宽度与 gamma 和标准误相关
    
    # 1. 计算 ATE 的标准误差
    n1 = (T == 1).sum()
    n0 = (T == 0).sum()
    var1 = Y[T == 1].var()
    var0 = Y[T == 0].var()
    
    se = None  # 你的代码: np.sqrt(var1/n1 + var0/n0)
    
    # 2. 边界宽度随 gamma 增加
    # 简化公式: bound_width = se * log(gamma) * 2
    bound_width = None  # 你的代码
    
    # 3. 计算上下界
    lower_bound = None  # 你的代码: ate_obs - bound_width
    upper_bound = None  # 你的代码: ate_obs + bound_width
    
    return lower_bound, upper_bound

In [ ]:
def sensitivity_curve(
    Y: np.ndarray,
    T: np.ndarray,
    gamma_range: np.ndarray,
    true_ate: float = None
) -> pd.DataFrame:
    """
    计算敏感性曲线
    """
    results = []
    
    for gamma in gamma_range:
        lower, upper = compute_rosenbaum_bounds(Y, T, gamma)
        
        row = {
            'gamma': gamma,
            'lower': lower,
            'upper': upper
        }
        
        if true_ate is not None:
            row['true_ate'] = true_ate
        
        results.append(row)
    
    return pd.DataFrame(results)

In [ ]:
# 生成有混淆的数据
df, U, params = simulate_unobserved_confounding(
    n=2000,
    confounder_strength=0.6
)

if df is not None and df['Y'].iloc[0] is not None:
    # 计算敏感性曲线
    gamma_range = np.linspace(1.0, 5.0, 30)
    
    try:
        sens_df = sensitivity_curve(
            df['Y'].values,
            df['T'].values,
            gamma_range,
            true_ate=params['true_ate']
        )
        
        if sens_df is not None and sens_df['lower'].iloc[0] is not None:
            print("敏感性分析结果:")
            print("="*60)
            print(f"\n观测 ATE: {df['Y'][df['T']==1].mean() - df['Y'][df['T']==0].mean():.4f}")
            print(f"真实 ATE: {params['true_ate']:.4f}")
            
            print("\n部分结果:")
            display(sens_df.iloc[::5].round(4))  # 每5行显示一次
            
            # 找到包含 0 的最小 gamma
            includes_zero = (sens_df['lower'] <= 0) & (sens_df['upper'] >= 0)
            if includes_zero.any():
                gamma_threshold = sens_df.loc[includes_zero, 'gamma'].min()
                print(f"\n敏感性阈值: Γ = {gamma_threshold:.2f}")
                print(f"解读: 如果未观测混淆使倾向得分相差 {gamma_threshold:.1f} 倍，")
                print(f"      效应可能变为零（不显著）")
            else:
                print(f"\n在 Γ ≤ {gamma_range.max():.1f} 范围内，效应始终显著")
                print("结论相对稳健！")
        else:
            print("[TODO] 请完成 compute_rosenbaum_bounds 函数")
    except Exception as e:
        print(f"[错误] {e}")
else:
    print("[TODO] 请完成数据生成")

In [ ]:
# 可视化敏感性分析
if 'sens_df' in dir() and sens_df is not None and sens_df['lower'].iloc[0] is not None:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # 绘制边界
    ax.fill_between(
        sens_df['gamma'], 
        sens_df['lower'], 
        sens_df['upper'],
        alpha=0.3, color='#3498db', label='敏感性区间'
    )
    ax.plot(sens_df['gamma'], sens_df['lower'], 'b-', linewidth=2, label='下界')
    ax.plot(sens_df['gamma'], sens_df['upper'], 'b-', linewidth=2, label='上界')
    
    # 真实 ATE
    ax.axhline(y=params['true_ate'], color='green', linestyle='--', 
              linewidth=2, label=f'真实 ATE = {params["true_ate"]}')
    
    # 零线
    ax.axhline(y=0, color='red', linestyle='-', linewidth=1, label='零效应')
    
    # 找到交叉点
    includes_zero = (sens_df['lower'] <= 0) & (sens_df['upper'] >= 0)
    if includes_zero.any():
        gamma_threshold = sens_df.loc[includes_zero, 'gamma'].min()
        ax.axvline(x=gamma_threshold, color='orange', linestyle=':', 
                  linewidth=2, label=f'敏感性阈值 Γ={gamma_threshold:.2f}')
    
    ax.set_xlabel('Γ (敏感性参数)', fontsize=12)
    ax.set_ylabel('ATE 估计', fontsize=12)
    ax.set_title('Rosenbaum 敏感性分析\n"结论对未观测混淆有多脆弱?"', fontsize=14)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n解读指南:")
    print("- Γ = 1: 假设无未观测混淆")
    print("- 随着 Γ 增加，允许更强的未观测混淆")
    print("- 当区间包含 0 时，结论变得不确定")
    print("- 敏感性阈值越高，结论越稳健")

## 练习 3.3: E-value - 更直观的敏感性度量

E-value 是由 VanderWeele & Ding (2017) 提出的一种流行的敏感性度量方法。

### 为什么需要 E-value？

**Rosenbaum 方法的局限**:
- 需要匹配设计
- Γ 参数的实际意义不够直观
- 难以解释给非技术人员

**E-value 的优势**:
- 适用于任何研究设计
- 直接量化「需要多强的混淆」
- 容易向利益相关者解释

### 核心问题

> **「需要多强的未观测混淆因子，才能完全解释掉观测到的关联？」**

### E-value 的直觉

假设你发现「培训使薪资提高 50%」(RR = 1.5):

- **E-value = 2.0**: 需要一个与处理和结果都有 2 倍关联的混淆因子
- **实际意义**: 如果存在一个未观测因子，它使「参加培训的概率」提高 2 倍，同时也使「薪资」提高 2 倍，那么观测到的效应可能完全是假的

**判断标准**:
- E-value 越大，结论越稳健
- E-value 很小 (< 1.5)，说明结论容易被推翻
- E-value 很大 (> 3)，说明需要很强的混淆才能推翻结论

In [ ]:
def ate_to_risk_ratio(ate: float, baseline_mean: float) -> float:
    """
    将 ATE 转换为风险比 (Risk Ratio)
    
    当结果变量是连续的时候，我们需要转换为风险比：
    RR = (baseline + ATE) / baseline
    
    示例：
    - 基线薪资: 10 万
    - ATE: 2 万
    - RR = (10 + 2) / 10 = 1.2  (薪资提高了 20%)
    
    Args:
        ate: 平均处理效应
        baseline_mean: 控制组的平均结果
    
    Returns:
        风险比
    """
    # TODO: 计算风险比
    # 你的代码
    pass


def compute_e_value(observed_rr: float, ci_lower: float = None) -> Dict[str, float]:
    """
    计算 E-value
    
    E-value: 使观测关联完全被混淆解释所需的最小风险比
    
    公式推导：
    假设存在未观测混淆因子 U，它与处理 T 和结果 Y 的风险比都是 RR_U。
    如果要完全解释观测到的 RR_obs，则：
    
    E = RR_obs + sqrt(RR_obs * (RR_obs - 1))  [当 RR >= 1]
    
    直觉理解：
    - E-value 是关于 RR 的递增函数
    - RR_obs 越大，E-value 越大，结论越稳健
    - E-value = 1 意味着任何混淆都能推翻结论（结论非常脆弱）
    
    Args:
        observed_rr: 观测到的风险比
        ci_lower: 置信区间下界 (可选)
    
    Returns:
        {'e_value': ..., 'e_value_ci': ...}
    """
    # 确保 RR >= 1 (如果 < 1，取倒数)
    if observed_rr < 1:
        observed_rr = 1 / observed_rr
    
    # TODO: 计算 E-value
    # E = RR + sqrt(RR * (RR - 1))
    e_value = None  # 你的代码
    
    result = {'e_value': e_value}
    
    # TODO: 如果有置信区间下界，也计算其 E-value
    # 这给出了更保守的估计
    if ci_lower is not None:
        if ci_lower < 1:
            ci_lower = 1 / ci_lower
        e_value_ci = None  # 你的代码: ci_lower + np.sqrt(ci_lower * (ci_lower - 1))
        result['e_value_ci'] = e_value_ci
    
    return result


def compute_ate_ci(Y: np.ndarray, T: np.ndarray, alpha: float = 0.05) -> Tuple[float, float, float]:
    """
    计算 ATE 的置信区间
    
    使用 t 检验计算
    
    Returns:
        (ate, ci_lower, ci_upper)
    """
    Y1 = Y[T == 1]
    Y0 = Y[T == 0]
    
    ate = Y1.mean() - Y0.mean()
    
    # TODO: 计算标准误
    n1, n0 = len(Y1), len(Y0)
    var1, var0 = Y1.var(ddof=1), Y0.var(ddof=1)
    
    se = None  # 你的代码: np.sqrt(var1/n1 + var0/n0)
    
    # TODO: 计算置信区间
    # 使用 t 分布的临界值
    from scipy import stats
    df = n1 + n0 - 2
    t_crit = None  # 你的代码: stats.t.ppf(1 - alpha/2, df)
    
    ci_lower = None  # 你的代码: ate - t_crit * se
    ci_upper = None  # 你的代码: ate + t_crit * se
    
    return ate, ci_lower, ci_upper

In [ ]:
# 测试 E-value 计算
print("=== E-value 分析示例 ===\n")

if df is not None and df['Y'].iloc[0] is not None:
    try:
        # 计算 ATE 和置信区间
        ate, ci_lower, ci_upper = compute_ate_ci(df['Y'].values, df['T'].values)
        
        if ate is not None and ci_lower is not None:
            baseline_mean = df.loc[df['T'] == 0, 'Y'].mean()
            
            # 转换为风险比
            rr = ate_to_risk_ratio(ate, baseline_mean)
            rr_ci_lower = ate_to_risk_ratio(ci_lower, baseline_mean)
            rr_ci_upper = ate_to_risk_ratio(ci_upper, baseline_mean)
            
            if rr is not None:
                print(f"步骤 1: 计算效应估计")
                print(f"  观测 ATE: {ate:.4f}")
                print(f"  95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
                print(f"  基线均值: {baseline_mean:.4f}")
                
                print(f"\n步骤 2: 转换为风险比")
                print(f"  风险比 (RR): {rr:.4f}")
                print(f"  95% CI: [{rr_ci_lower:.4f}, {rr_ci_upper:.4f}]")
                
                # 计算 E-value
                e_values = compute_e_value(rr, rr_ci_lower)
                
                if e_values['e_value'] is not None:
                    print(f"\n步骤 3: 计算 E-value")
                    print(f"  点估计的 E-value: {e_values['e_value']:.4f}")
                    
                    if 'e_value_ci' in e_values and e_values['e_value_ci'] is not None:
                        print(f"  置信区间下界的 E-value: {e_values['e_value_ci']:.4f}")
                    
                    print(f"\n步骤 4: 解读 E-value")
                    print("="*60)
                    print(f"\n点估计解读:")
                    print(f"  需要一个未观测混淆因子 U，它同时满足:")
                    print(f"    • 使参加培训的概率提高 {e_values['e_value']:.2f} 倍")
                    print(f"    • 使薪资提高 {e_values['e_value']:.2f} 倍")
                    print(f"  才能完全解释掉观测到的 {rr:.2f} 倍效应")
                    
                    if 'e_value_ci' in e_values and e_values['e_value_ci'] is not None:
                        print(f"\n置信区间解读 (更保守):")
                        print(f"  即使考虑统计不确定性，也需要 {e_values['e_value_ci']:.2f} 倍的")
                        print(f"  混淆关联才能使效应变为零")
                    
                    # 稳健性评价
                    print(f"\n稳健性评价:")
                    if e_values['e_value'] < 1.5:
                        print("  ⚠️ E-value < 1.5: 结论非常脆弱")
                        print("     即使较弱的混淆也可能推翻结论")
                    elif e_values['e_value'] < 2.5:
                        print("  ⚠️ E-value 在 1.5-2.5: 结论中等稳健")
                        print("     需要考虑可能的混淆因子")
                    elif e_values['e_value'] < 4.0:
                        print("  ✅ E-value 在 2.5-4.0: 结论较为稳健")
                        print("     需要较强的混淆才能推翻结论")
                    else:
                        print("  ✅✅ E-value > 4.0: 结论非常稳健")
                        print("     需要非常强的混淆才能推翻结论")
                    
                    print(f"\n实际判断:")
                    print(f"  问自己: 是否存在一个未观测因子，能同时与")
                    print(f"  处理和结果都有 {e_values['e_value']:.1f} 倍的关联?")
                    print(f"  如果很难想象这样的因子，那么结论是稳健的！")
                else:
                    print("[TODO] 请完成 compute_e_value 函数")
            else:
                print("[TODO] 请完成 ate_to_risk_ratio 函数")
        else:
            print("[TODO] 请完成 compute_ate_ci 函数")
    except Exception as e:
        print(f"[错误] {e}")
else:
    print("[TODO] 请先完成数据生成")

### 如何在论文中报告 E-value

E-value 已经成为顶级期刊（如 JAMA, The Lancet, Epidemiology）推荐的敏感性分析方法。

#### 报告模板

**情况 1: E-value 较高（稳健）**

> "We observed a risk ratio of 1.80 (95% CI: 1.45-2.20) for the association between [treatment] and [outcome]. The E-value for the point estimate was 3.0, and the E-value for the lower confidence limit was 2.3. This suggests that an unmeasured confounder would need to be associated with both [treatment] and [outcome] by a risk ratio of at least 3.0-fold each, above and beyond the measured confounders, to fully explain away the observed association. Such strong confounding seems unlikely given the measured covariates already controlled for."

中文版:
> "我们观察到 [处理] 与 [结果] 的风险比为 1.80 (95% CI: 1.45-2.20)。点估计的 E-value 为 3.0，置信区间下界的 E-value 为 2.3。这意味着，在已经控制的混淆因子之外，还需要一个未观测混淆因子与处理和结果都有至少 3.0 倍的关联，才能完全解释掉观测到的关联。考虑到我们已经控制了主要混淆因子，如此强的混淆不太可能存在。"

**情况 2: E-value 较低（需要谨慎）**

> "The E-value for our point estimate was 1.8, suggesting that moderate unmeasured confounding could potentially explain the observed association. We acknowledge this as a limitation and recommend cautious interpretation of our findings."

中文版:
> "我们点估计的 E-value 为 1.8，说明中等强度的未观测混淆就可能解释观测到的关联。我们承认这是研究的局限性，建议谨慎解读结果。"

#### 什么时候报告 E-value？

✅ **应该报告的情况**:
1. 观察性研究（非随机实验）
2. 无法控制所有重要混淆因子
3. 结论有重要政策或临床意义
4. 审稿人或期刊要求

❌ **不需要报告的情况**:
1. 随机对照实验 (RCT)
2. 工具变量 (IV) 研究（已经处理了混淆）
3. 结果已经不显著（E-value 没有意义）

#### E-value 的局限性

⚠️ **重要警告**:

1. **E-value 不能证明无混淆**: E-value 高只说明需要强混淆才能推翻结论，不代表不存在混淆
2. **对效应大小敏感**: 效应越大，E-value 越大，但效应大可能本身就是因为混淆
3. **假设简化**: E-value 假设混淆因子与处理、结果的关联强度相同，实际可能不同
4. **不能替代领域知识**: 需要结合专业知识判断「这样的混淆因子是否合理存在」

#### 实际案例

**案例 1: 吸烟与肺癌**
- 观测 RR ≈ 10
- E-value ≈ 19
- 解读: 需要一个与吸烟和肺癌都有 19 倍关联的混淆因子。很难想象这样的因子！

**案例 2: 培训与薪资**
- 观测 RR = 1.2
- E-value ≈ 1.6
- 解读: 「动机」这个未观测因子很可能同时影响培训和薪资，且关联强度可能超过 1.6 倍。结论脆弱！

In [ ]:
# 对比不同混淆强度下的 E-value
print("=== 不同混淆强度下的 E-value 对比 ===\n")

comparison_results = []

for strength in [0.0, 0.3, 0.6, 1.0]:
    df_temp, U_temp, params_temp = simulate_unobserved_confounding(
        n=2000,
        confounder_strength=strength,
        seed=42 + int(strength * 10)
    )
    
    if df_temp is not None and df_temp['Y'].iloc[0] is not None:
        try:
            # 朴素估计
            naive_ate = df_temp['Y'][df_temp['T']==1].mean() - df_temp['Y'][df_temp['T']==0].mean()
            baseline = df_temp.loc[df_temp['T'] == 0, 'Y'].mean()
            
            # 计算置信区间
            ate, ci_lower, ci_upper = compute_ate_ci(
                df_temp['Y'].values, 
                df_temp['T'].values
            )
            
            if ate is not None and ci_lower is not None:
                # 转换为风险比
                rr = ate_to_risk_ratio(ate, baseline)
                rr_ci_lower = ate_to_risk_ratio(ci_lower, baseline)
                
                if rr is not None:
                    # 计算 E-value
                    e_vals = compute_e_value(rr, rr_ci_lower)
                    
                    if e_vals['e_value'] is not None:
                        comparison_results.append({
                            '混淆强度': strength,
                            'ATE (真实=2.0)': ate,
                            '偏差': ate - 2.0,
                            '风险比': rr,
                            'E-value (点估计)': e_vals['e_value'],
                            'E-value (CI下界)': e_vals.get('e_value_ci', np.nan)
                        })
        except:
            pass

if len(comparison_results) > 0:
    comp_df = pd.DataFrame(comparison_results)
    display(comp_df.round(4))
    
    print("\n关键洞察:")
    print("1. 混淆强度 = 0: 无偏估计，E-value 较高 (稳健)")
    print("2. 混淆强度增加: 估计偏差增大，但 E-value 也可能增大")
    print("3. E-value 高不代表估计无偏！只表示「如果要推翻结论，需要多强的混淆」")
    print("4. 这就是为什么 E-value 是「敏感性分析」而非「验证无偏性」")
else:
    print("[TODO] 请完成上面的函数")

In [ ]:
# 可视化 E-value 与风险比的关系
def plot_evalue_curve():
    """
    绘制 E-value 与风险比的关系曲线
    帮助理解不同效应大小对应的 E-value
    """
    rr_range = np.linspace(1.0, 5.0, 100)
    e_values = rr_range + np.sqrt(rr_range * (rr_range - 1))
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图: E-value vs RR
    axes[0].plot(rr_range, e_values, 'b-', linewidth=2)
    axes[0].axhline(y=1.5, color='red', linestyle='--', alpha=0.5, label='E=1.5 (脆弱)')
    axes[0].axhline(y=2.5, color='orange', linestyle='--', alpha=0.5, label='E=2.5 (中等)')
    axes[0].axhline(y=4.0, color='green', linestyle='--', alpha=0.5, label='E=4.0 (稳健)')
    axes[0].fill_between(rr_range, 0, 1.5, alpha=0.1, color='red')
    axes[0].fill_between(rr_range, 1.5, 2.5, alpha=0.1, color='orange')
    axes[0].fill_between(rr_range, 2.5, 4.0, alpha=0.1, color='yellow')
    axes[0].fill_between(rr_range, 4.0, e_values.max(), alpha=0.1, color='green')
    
    # 标注实际案例
    if 'rr' in dir() and rr is not None and e_values is not None:
        try:
            current_evalue = compute_e_value(rr)
            if current_evalue['e_value'] is not None:
                axes[0].scatter([rr], [current_evalue['e_value']], 
                              s=200, color='darkblue', zorder=5, 
                              label=f'当前研究 (RR={rr:.2f}, E={current_evalue["e_value"]:.2f})')
        except:
            pass
    
    axes[0].set_xlabel('观测到的风险比 (RR)', fontsize=12)
    axes[0].set_ylabel('E-value', fontsize=12)
    axes[0].set_title('E-value 与风险比的关系\n"效应越大，结论越稳健"', fontsize=13)
    axes[0].legend(loc='upper left')
    axes[0].grid(True, alpha=0.3)
    
    # 右图: 需要的混淆强度示意图
    example_rrs = [1.5, 2.0, 3.0, 4.0]
    example_evalues = [rr + np.sqrt(rr * (rr - 1)) for rr in example_rrs]
    
    colors = ['#e74c3c', '#e67e22', '#f39c12', '#27ae60']
    bars = axes[1].barh(range(len(example_rrs)), example_evalues, color=colors, alpha=0.7)
    
    axes[1].set_yticks(range(len(example_rrs)))
    axes[1].set_yticklabels([f'RR = {rr:.1f}' for rr in example_rrs])
    axes[1].set_xlabel('需要的混淆强度 (E-value)', fontsize=12)
    axes[1].set_title('不同效应大小对应的稳健性要求', fontsize=13)
    axes[1].grid(True, alpha=0.3, axis='x')
    
    # 添加数值标注
    for i, (rr, ev) in enumerate(zip(example_rrs, example_evalues)):
        axes[1].text(ev + 0.1, i, f'E = {ev:.2f}', 
                    va='center', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# 绘制曲线
try:
    plot_evalue_curve()
    
    print("\n图表解读:")
    print("左图: E-value 随风险比单调递增")
    print("  - 效应越大 (RR 越大)，需要越强的混淆才能推翻")
    print("  - 颜色分区显示不同的稳健性等级")
    print("\n右图: 具体案例对比")
    print("  - RR=1.5 → E=2.0: 需要 2 倍的混淆")
    print("  - RR=3.0 → E=4.7: 需要 4.7 倍的混淆 (很稳健!)")
except Exception as e:
    print(f"[提示] 图表需要完成上面的函数才能显示: {e}")

## 练习 3.4: Placebo 稳健性检验

Placebo 测试是一种简单但强大的稳健性检验：

**思路**: 如果我们的因果估计方法是正确的，那么对于「不应受处理影响的变量」，我们不应该检测到显著效应。

例如：
- 研究「培训」对「薪资」的影响
- Placebo 变量：参加培训前的身高（显然培训不应该改变身高）
- 如果我们在身高上也发现了「效应」，说明可能有问题！

In [ ]:
def placebo_test(
    df: pd.DataFrame,
    outcome_col: str = 'Y',
    placebo_outcome_col: str = 'Y_placebo'
) -> Dict[str, float]:
    """
    Placebo 测试
    
    使用不应受处理影响的"假"结果变量进行测试
    
    Args:
        df: 包含 T, Y, Y_placebo 的 DataFrame
        outcome_col: 真实结果列名
        placebo_outcome_col: 假结果列名
    
    Returns:
        {'true_effect': ..., 'placebo_effect': ..., 'p_value_placebo': ...}
    """
    T = df['T'].values
    
    # TODO: 计算真实结果的效应
    Y = df[outcome_col].values
    true_effect = None  # 你的代码: Y[T==1].mean() - Y[T==0].mean()
    
    # TODO: 计算假结果的效应
    Y_placebo = df[placebo_outcome_col].values
    placebo_effect = None  # 你的代码
    
    # TODO: t 检验测试假结果效应是否显著
    # 使用 scipy.stats.ttest_ind
    t_stat, p_value = None, None  # 你的代码
    
    return {
        'true_effect': true_effect,
        'placebo_effect': placebo_effect,
        'p_value_placebo': p_value
    }

In [ ]:
# 测试 Placebo 检验
if df is not None and df['Y'].iloc[0] is not None:
    # 生成假结果变量（不应受 T 影响，只与 X 相关）
    np.random.seed(123)
    df['Y_placebo'] = df['X'] * 2 + np.random.randn(len(df)) * 0.5
    
    try:
        placebo_results = placebo_test(df, 'Y', 'Y_placebo')
        
        if placebo_results['true_effect'] is not None:
            print("Placebo 测试结果:")
            print("="*50)
            print(f"\n真实结果 (Y) 的效应: {placebo_results['true_effect']:.4f}")
            print(f"假结果 (Y_placebo) 的效应: {placebo_results['placebo_effect']:.4f}")
            
            if placebo_results['p_value_placebo'] is not None:
                print(f"假结果 p-value: {placebo_results['p_value_placebo']:.4f}")
                
                print("\n解读:")
                if placebo_results['p_value_placebo'] < 0.05:
                    print("  ⚠️ 警告: 假结果也显示显著效应!")
                    print("  这可能意味着:")
                    print("  - 存在未控制的混淆")
                    print("  - 选择偏差问题")
                    print("  - 模型设定错误")
                else:
                    print("  ✅ 假结果无显著效应，通过 Placebo 测试!")
                    print("  这增强了我们对因果估计的信心")
        else:
            print("[TODO] 请完成 placebo_test 函数")
    except Exception as e:
        print(f"[错误] {e}")

In [ ]:
# 对比有混淆和无混淆情况下的 Placebo 测试
print("对比不同混淆强度下的 Placebo 测试:")
print("="*60)

placebo_comparison = []

for strength in [0.0, 0.3, 0.6, 1.0]:
    df_temp, U_temp, params_temp = simulate_unobserved_confounding(
        n=2000,
        confounder_strength=strength
    )
    
    if df_temp is not None and df_temp['Y'].iloc[0] is not None:
        # 生成依赖于 U 的假结果（如果 U 是混淆，这个也会被"影响"）
        df_temp['Y_placebo'] = U_temp * 2 + np.random.randn(len(df_temp)) * 0.5
        
        results = placebo_test(df_temp, 'Y', 'Y_placebo')
        
        if results['placebo_effect'] is not None:
            placebo_comparison.append({
                'Strength': strength,
                'True Effect': results['true_effect'],
                'Placebo Effect': results['placebo_effect'],
                'P-value': results['p_value_placebo']
            })

if len(placebo_comparison) > 0:
    comp_df = pd.DataFrame(placebo_comparison)
    display(comp_df.round(4))
    
    print("\n解读:")
    print("- 当混淆强度为 0 时，Placebo 效应接近 0")
    print("- 随着混淆强度增加，Placebo 效应变得显著")
    print("- 这说明 Placebo 测试可以帮助检测混淆!")

# 思考题 3: E-value 和 Rosenbaum bounds 有何异同？

answer_3 = """
你的答案:

相同点:
- 

不同点:
- 

各自适用场景:
- 

"""
print(answer_3)

In [ ]:
# 思考题 1: 为什么无混淆假设无法从数据中验证？

answer_1 = """
你的答案:



"""
print(answer_1)

In [ ]:
# 思考题 2: Rosenbaum Γ = 2 意味着什么？在实践中这算强混淆还是弱混淆？

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 9: E-value 很高是否意味着因果关系一定成立？

answer_9 = """
你的答案:



"""
print(answer_9)

In [ ]:
# 思考题 8: 为什么 E-value 要同时计算点估计和置信区间下界的值？

answer_8 = """
你的答案:



"""
print(answer_8)

In [ ]:
# 思考题 7: E-value = 2.5 在实践中算高还是低？能举一个现实中的混淆因子例子吗？

answer_7 = """
你的答案:



"""
print(answer_7)

In [ ]:
# 思考题 3: E-value 和 Rosenbaum bounds 有何异同？

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
## 📊 总结

### 敏感性分析方法对比

| 方法 | 回答的问题 | 优点 | 缺点 | 适用场景 |
|------|------------|------|------|---------|
| **Rosenbaum Bounds** | 多强的混淆会推翻结论？| 有严格理论基础 | 需要匹配设计；Γ 不够直观 | 倾向得分匹配后 |
| **E-value** | 解释效应需要多强的混淆？| 简单计算；易于解释；广泛适用 | 假设简化；不能证明因果 | 任何观察性研究 |
| **Placebo Test** | 方法是否可靠？| 直接检验假设；易于理解 | 需要好的 placebo 变量 | 有合适 placebo 时 |

### E-value 解读指南

| E-value 范围 | 稳健性评价 | 实际含义 | 建议 |
|-------------|-----------|---------|------|
| < 1.5 | ❌ 非常脆弱 | 弱混淆即可推翻 | 谨慎解释，承认局限 |
| 1.5 - 2.0 | ⚠️ 较为脆弱 | 中等混淆可能推翻 | 寻找更多证据 |
| 2.0 - 2.5 | 🟡 中等稳健 | 需要较强混淆 | 结合领域知识判断 |
| 2.5 - 4.0 | ✅ 较为稳健 | 需要强混淆 | 可以有一定信心 |
| > 4.0 | ✅✅ 非常稳健 | 需要极强混淆 | 结论可信度高 |

**关键提醒**: E-value 高不代表无混淆，只表示需要多强的混淆才能推翻结论！

### E-value 公式速查

对于不同的效应度量，E-value 的计算公式：

**1. 风险比 (Risk Ratio, RR)**
```
E = RR + sqrt(RR * (RR - 1))   [当 RR ≥ 1]
E = 1/RR + sqrt(1/RR * (1/RR - 1))   [当 RR < 1]
```

**2. 比值比 (Odds Ratio, OR)** - 对于罕见结果
```
E ≈ OR + sqrt(OR * (OR - 1))   [当 OR ≥ 1]
```

**3. 标准化均值差 (Cohen's d)**
```
RR_approx = exp(0.91 * d)
然后计算 E-value
```

### 何时使用敏感性分析？

✅ **总是应该做**:
- 观察性研究
- 无法随机化的场景
- 结论有重要政策/临床意义
- 可能存在未观测混淆

⚠️ **谨慎使用**:
- 随机对照实验（已经处理了混淆）
- 工具变量研究（有其他方法处理混淆）

### 敏感性分析的正确心态

**不是为了**:
- ❌ 证明因果关系成立
- ❌ 验证无混淆假设
- ❌ 替代随机实验

**而是为了**:
- ✅ 评估结论的稳健性
- ✅ 量化不确定性
- ✅ 指导后续研究方向
- ✅ 诚实地讨论局限性

### 最佳实践

1. **报告多个指标**: 同时报告点估计和置信区间的 E-value
2. **结合领域知识**: 思考「这样的混淆因子是否合理存在」
3. **使用多种方法**: Rosenbaum + E-value + Placebo 交叉验证
4. **诚实讨论**: 在论文的 Limitation 部分坦诚讨论
5. **考虑方向**: 未观测混淆可能高估也可能低估效应

In [ ]:
# 思考题 5: 如果敏感性分析显示结论对未观测混淆很敏感，应该怎么办？

answer_5 = """
你的答案:



"""
print(answer_5)

In [ ]:
# 思考题 6: Placebo 测试的原理是什么？它能检测哪种类型的偏差？

answer_6 = """
你的答案:



"""
print(answer_6)

## 📊 总结

### 敏感性分析方法对比

| 方法 | 回答的问题 | 优点 | 缺点 |
|------|------------|------|------|
| Rosenbaum Bounds | 多强的混淆会推翻结论？ | 直观、有理论支撑 | 需要匹配 |
| E-value | 解释效应需要多强的混淆？ | 简单计算、广泛适用 | 假设较强 |
| Placebo Test | 方法是否可靠？ | 直接检验假设 | 需要好的 placebo 变量 |

### 何时使用敏感性分析？

✅ **总是应该做**:
- 观察性研究
- 无法随机化的情况
- 结论有重要政策含义

### 如何解读结果？

| E-value / Γ 阈值 | 解读 |
|------------------|------|
| < 1.5 | 结论非常脆弱 |
| 1.5 - 2.5 | 中等稳健 |
| 2.5 - 4.0 | 较为稳健 |
| > 4.0 | 非常稳健 |

### 最佳实践

1. **总是报告敏感性分析**
2. **结合领域知识**判断假设的混淆是否合理
3. **使用多种方法**交叉验证
4. **诚实地讨论局限性**

In [ ]:
print("🔍 恭喜完成敏感性分析练习!")
print("\n你已经学会了:")
print("  ✓ 理解未观测混淆的影响")
print("  ✓ Rosenbaum 敏感性边界")
print("  ✓ E-value 计算和解读")
print("  ✓ Placebo 稳健性检验")
print("\n恭喜完成 Chapter 5 所有练习!")
print("下一步: Chapter 6 应用场景 - 优惠券优化!")